In [5]:
import os
import requests
import pandas as pd
import json
import csv

In [6]:
csv_url = "https://ogd.parl.apps.be.ch/data/abstimmungsliste.csv"
csv_filename = "abstimmungsliste.csv"

if not os.path.exists(csv_filename):
    response = requests.get(csv_url)
    response.raise_for_status()
    with open(csv_filename, "wb") as f:
        f.write(response.content)

In [7]:
dataframe = pd.read_csv(csv_filename)

display(dataframe.head(5))

,abstimmungsliste_uid,abstimmung_uid,traktandum_uid,mitglied_uid,mitglied_vorname,mitglied_name,abstimmungsliste_stimme,Übersicht vom
0,4d5daac7-de8e-463e-a761-3074675b55e3,0cb9793f-cb05-4f59-9576-bfe4c329ea02,a3d0be7d3ca2411db7df770f687536a7,c310853a6cf740ecaa932248726d47ba,Madeleine,Amstutz,Nein,2025-05-19 01:30
1,484bac5f-c221-452b-9d42-daa51f1e0025,0cb9793f-cb05-4f59-9576-bfe4c329ea02,a3d0be7d3ca2411db7df770f687536a7,c2a1d34af2da4d9a9010abb9f27e486b,Jakob,Schwarz,Nein,2025-05-19 01:30
2,71f55319-6ce6-4eec-85e6-5e8c05c85fb2,0cb9793f-cb05-4f59-9576-bfe4c329ea02,a3d0be7d3ca2411db7df770f687536a7,58b290b0fb5448ca97e90bfb10e3cb8a,Barbara,Maurer,Nein,2025-05-19 01:30
3,00b33f14-27b1-4927-a0d5-f7edbe0bbae5,0cb9793f-cb05-4f59-9576-bfe4c329ea02,a3d0be7d3ca2411db7df770f687536a7,88d8b686b32746aa89fc69eac570d37c,Bruno,Martin,Nein,2025-05-19 01:30
4,431bd498-e591-4fbb-8816-d9cd94530932,0cb9793f-cb05-4f59-9576-bfe4c329ea02,a3d0be7d3ca2411db7df770f687536a7,3567f54e8b2d4df0912ce8f73f65c2ab,Samuel,Kullmann,Nein,2025-05-19 01:30


In [8]:
def vote_to_int(vote):
    if vote == "Enthalten":
        return 0
    elif vote == "Ja":
        return 1
    elif vote == "Nein":
        return 2
    else:
        return -1

# Get all unique abstimmung_uid values in the order they appear
abstimmung_uids = list(dict.fromkeys(dataframe["abstimmung_uid"]))

# Prepare header for CSV
header = ["id", "first_name", "last_name"] + abstimmung_uids

# Prepare a mapping from mitglied_uid to their row data
rows = {}
for _, row in dataframe.iterrows():
    mitglied_uid = row["mitglied_uid"]
    if mitglied_uid not in rows:
        rows[mitglied_uid] = {
            "id": mitglied_uid,
            "first_name": row["mitglied_vorname"],
            "last_name": row["mitglied_name"],
            **{uid: -1 for uid in abstimmung_uids},  # fill with -1 by default
        }
    abstimmung_uid = row["abstimmung_uid"]
    stimme = vote_to_int(row["abstimmungsliste_stimme"])
    rows[mitglied_uid][abstimmung_uid] = stimme

# Write to CSV
with open("votes.csv", "w", encoding="utf-8", newline="") as f:
    writer = csv.DictWriter(f, fieldnames=header)
    writer.writeheader()
    for row in rows.values():
        writer.writerow(row)
print("Result written to votes.csv")


Result written to votes.csv
